# CMOR 461 - Homework 7  
**Warren Weissbluth**  

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wweissbluth/CMOR-461-LOGISTICS-AND-SUPPLY-CHAIN-MANAGEMENT/blob/main/hw7/hw7.ipynb)

---

## 1) Steelworks Case 

### Problem Statement  

Consider the **Steel Works** case study discussed in class. Assume that a **base-stock policy** is used to manage product inventories. Your task is to:  

1. **Calculate**:
   - Safety stock  
   - Base-stock level  
   - Average inventory level  

2. **Compare** the computed average inventory levels with those provided in the case study.  

3. **Assumptions**:
   - **Review period** ($ R $) = 1 month  
   - **Lead time** ($ L $) = 0.25 and 0.5 months  
   - **Service levels** ($\alpha$) = 0.90 and 0.97  

4. **Summarize** your recommendations for Steel Works based on your findings.  

### Base Stock Policy

The base stock policy is defined by:
* Periodid Review (as opposed to continous)
* Has an optimal / target inventory level: the base stock level
* At each review, consider the current inventory position and then order more inventory to raise the current position to base stock level.

Vars:
* R = len of review period
* L = lead time
* $\mu$ = average demand
* $\sigma$ = SD of demand 
* S = base stock level

We need to ensure our stock covers R+L days. This is made up of two terms:
* Cycle Stock - $(R+L)\mu$
* Safety Stock - $\mathbb z\sigma\sqrt{R+L}$

Let's get those here. We need $\mu,\sigma$. We'll need this from the Steelworks Case


In [51]:
import pandas as pd

# Load the Excel file
file_path = '4.3.1.SteelWorksData.xlsx'
xls = pd.ExcelFile(file_path)

# Create a dictionary to store DataFrames
dfs = {}

# Display each sheet as a DataFrame with headers on row 3 (index 2)
for sheet_name in xls.sheet_names:
    if sheet_name == 'fincldat' or sheet_name == 'AVEINV':
        df = pd.read_excel(xls, sheet_name, header=1)
    else:
        df = pd.read_excel(xls, sheet_name, header=2)
    dfs[sheet_name] = df
    print(f"Sheet name: {sheet_name}")
    display(df)

Sheet name: Annual Sales


,SKU,LABEL,Month,Customer,Demand
0,S0121958,DURABEND R12,1,194,756
1,S0121958,DURABEND R12,2,194,1202
2,S0121958,DURABEND R12,3,194,929
3,S0121958,DURABEND R12,4,194,825
4,S0121958,DURABEND R12,5,194,1078
...,...,...,...,...,...
67,S0121958,DURABEND R12,8,445,3
68,S0121958,DURABEND R12,9,445,0
69,S0121958,DURABEND R12,10,445,10
70,S0121958,DURABEND R12,11,445,23


Sheet name: monthvol


,SKU,LABEL,MTH,Total Demand
0,S0101958,DURABEND R10,1,13
1,S0101958,DURABEND R10,2,28
2,S0101958,DURABEND R10,3,8
3,S0101958,DURABEND R10,4,14
4,S0101958,DURABEND R10,5,27
...,...,...,...,...
79,S0232758,DURAFLEX R23,8,63
80,S0232758,DURAFLEX R23,9,0
81,S0232758,DURAFLEX R23,10,1
82,S0232758,DURAFLEX R23,11,0


Sheet name: prodbat


...

Sheet name: fincldat


,SKU,LABEL,Unit Cost,Selling Price
0,S0101958,DURABEND R10,0.23,1.150
1,S0121958,DURABEND R12,0.25,1.000
2,S0151958,DURABEND R15,0.27,1.026
3,S0102758,DURAFLEX R10,0.17,0.680
4,S0122758,DURAFLEX R12,0.22,0.880
5,S0152758,DURAFLEX R15,0.30,1.140
6,S0232758,DURAFLEX R23,0.31,1.550


Sheet name: AVEINV


/opt/homebrew/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,SKU,LABEL,Month,Average inventory over the month
0,S0101958,DURABEND R10,1,74.61
1,S0101958,DURABEND R10,2,68.94
2,S0101958,DURABEND R10,3,76.38
3,S0101958,DURABEND R10,4,67.84
4,S0101958,DURABEND R10,5,68.06
...,...,...,...,...
79,S0232758,DURAFLEX R23,8,191.46
80,S0232758,DURAFLEX R23,9,187.29
81,S0232758,DURAFLEX R23,10,182.06
82,S0232758,DURAFLEX R23,11,190.02


## $\mu$ and $\sigma$

We are to do the following for all products in the 'monthvol' sheet.

In [52]:
# Group by 'LABEL' and calculate mean and standard deviation of 'Total Demand'
monthvol_df = dfs['monthvol']
demand_stats = monthvol_df.groupby('LABEL')['Total Demand'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
demand_stats.columns = ['LABEL', 'Mean Demand', 'Standard Deviation']

# Display the resulting DataFrame
display(demand_stats)

,LABEL,Mean Demand,Standard Deviation
0,DURABEND R10,15.583333,13.187172
1,DURABEND R12,1008.000000,255.917245
2,DURABEND R15,2463.583333,494.209462
3,DURAFLEX R10,97.250000,92.524321
4,DURAFLEX R12,18.583333,11.429295
5,DURAFLEX R15,54.833333,80.054338
6,DURAFLEX R23,35.583333,45.862360


We need to ensure our stock covers R+L days. This is made up of two terms:
* Cycle Stock - $(R+L)\mu$
* Safety Stock - $\mathbb z\sigma\sqrt{R+L}$
* Average Inventory - $\frac{R\mu}{2} + z\sigma \sqrt{R + L}.$

In [53]:
def q1():
    from scipy.stats import norm

    R=1
    L=(.25,.5)
    alpha = (.9,.97)

    for label, row in demand_stats.iterrows():
        mean_demand = row['Mean Demand']
        stdev_demand = row['Standard Deviation']
        scenario = 1
        for i in range(2):
            for j in range(2):
                z = norm.ppf(alpha[i])
                CS = (R + L[j]) * mean_demand
                SS = z * stdev_demand * (R + L[j])**0.5
                BS = CS + SS
                AI = R*mean_demand/2+z*stdev_demand*(R+L[j])**0.5
                demand_stats.at[label, f'L_{i+1}'] = L[i]
                demand_stats.at[label, f'alpha_{i+1}'] = alpha[i]
                demand_stats.at[label, f'CS_{scenario}'] = CS
                demand_stats.at[label, f'SS_{scenario}'] = SS
                demand_stats.at[label, f'BS_{scenario}'] = BS
                demand_stats.at[label, f'AvgInv_{scenario}'] = AI

                scenario += 1

    # Display each scenario in a separate table
    for scenario in range(1, 5):
        scenario_columns = ['LABEL', f'L_{(scenario-1)//2 + 1}', f'alpha_{(scenario-1)%2 + 1}', f'CS_{scenario}', f'SS_{scenario}', f'BS_{scenario}', f'AvgInv_{scenario}']
        print(f"Scenario {scenario}")
        display(demand_stats[scenario_columns])
q1()

Scenario 1


,LABEL,L_1,alpha_1,CS_1,SS_1,BS_1,AvgInv_1
0,DURABEND R10,0.25,0.9,19.479167,18.894821,38.373987,26.686487
1,DURABEND R12,0.25,0.9,1260.000000,366.682889,1626.682889,870.682889
2,DURABEND R15,0.25,0.9,3079.479167,708.112316,3787.591483,1939.903983
3,DURAFLEX R10,0.25,0.9,121.562500,132.570532,254.133032,181.195532
4,DURAFLEX R12,0.25,0.9,23.229167,16.376102,39.605269,25.667769
5,DURAFLEX R15,0.25,0.9,68.541667,114.703313,183.244979,142.119979
6,DURAFLEX R23,0.25,0.9,44.479167,65.712424,110.191591,83.504091


Scenario 2


,LABEL,L_1,alpha_2,CS_2,SS_2,BS_2,AvgInv_2
0,DURABEND R10,0.25,0.97,23.375,20.698239,44.073239,28.489906
1,DURABEND R12,0.25,0.97,1512.000,401.680979,1913.680979,905.680979
2,DURABEND R15,0.25,0.97,3695.375,775.698178,4471.073178,2007.489844
3,DURAFLEX R10,0.25,0.97,145.875,145.223742,291.098742,193.848742
4,DURAFLEX R12,0.25,0.97,27.875,17.939121,45.814121,27.230788
5,DURAFLEX R15,0.25,0.97,82.250,125.651183,207.901183,153.067850
6,DURAFLEX R23,0.25,0.97,53.375,71.984354,125.359354,89.776021


Scenario 3


,LABEL,L_2,alpha_1,CS_3,SS_3,BS_3,AvgInv_3
0,DURABEND R10,0.5,0.9,19.479167,27.729870,47.209036,35.521536
1,DURABEND R12,0.5,0.9,1260.000000,538.140526,1798.140526,1042.140526
2,DURABEND R15,0.5,0.9,3079.479167,1039.219298,4118.698465,2271.010965
3,DURAFLEX R10,0.5,0.9,121.562500,194.559326,316.121826,243.184326
4,DURAFLEX R12,0.5,0.9,23.229167,24.033421,47.262588,33.325088
5,DURAFLEX R15,0.5,0.9,68.541667,168.337555,236.879222,195.754222
6,DURAFLEX R23,0.5,0.9,44.479167,96.438966,140.918133,114.230633


Scenario 4


,LABEL,L_2,alpha_2,CS_4,SS_4,BS_4,AvgInv_4
0,DURABEND R10,0.5,0.97,23.375,30.376550,53.751550,38.168217
1,DURABEND R12,0.5,0.97,1512.000,589.503410,2101.503410,1093.503410
2,DURABEND R15,0.5,0.97,3695.375,1138.407703,4833.782703,2370.199370
3,DURAFLEX R10,0.5,0.97,145.875,213.129064,359.004064,261.754064
4,DURAFLEX R12,0.5,0.97,27.875,26.327294,54.202294,35.618960
5,DURAFLEX R15,0.5,0.97,82.250,184.404552,266.654552,211.821219
6,DURAFLEX R23,0.5,0.97,53.375,105.643594,159.018594,123.435261


To summarize above, here's our reccomended inventory levels to stock the facility:

In [54]:
summary_demand_stats = demand_stats[['LABEL'] + [f'BS_{i}' for i in range(1, 5)]]
display(summary_demand_stats)

,LABEL,BS_1,BS_2,BS_3,BS_4
0,DURABEND R10,38.373987,44.073239,47.209036,53.751550
1,DURABEND R12,1626.682889,1913.680979,1798.140526,2101.503410
2,DURABEND R15,3787.591483,4471.073178,4118.698465,4833.782703
3,DURAFLEX R10,254.133032,291.098742,316.121826,359.004064
4,DURAFLEX R12,39.605269,45.814121,47.262588,54.202294
5,DURAFLEX R15,183.244979,207.901183,236.879222,266.654552
6,DURAFLEX R23,110.191591,125.359354,140.918133,159.018594


In [55]:
summary_demand_stats = demand_stats[['LABEL'] + [f'AvgInv_{i}' for i in range(1, 5)]]
display(summary_demand_stats)

,LABEL,AvgInv_1,AvgInv_2,AvgInv_3,AvgInv_4
0,DURABEND R10,26.686487,28.489906,35.521536,38.168217
1,DURABEND R12,870.682889,905.680979,1042.140526,1093.503410
2,DURABEND R15,1939.903983,2007.489844,2271.010965,2370.199370
3,DURAFLEX R10,181.195532,193.848742,243.184326,261.754064
4,DURAFLEX R12,25.667769,27.230788,33.325088,35.618960
5,DURAFLEX R15,142.119979,153.067850,195.754222,211.821219
6,DURAFLEX R23,83.504091,89.776021,114.230633,123.435261


### Compare to Steelworks Case

Here's the actual values that the company's inventory is held at.

In [56]:
# Filter the AVEINV DataFrame for DURABEND R12
durabend_r12_aveinv_df = dfs['AVEINV'].groupby('LABEL')['Average inventory '
'over the month'].mean().reset_index()
durabend_r12_aveinv_df.columns = ['LABEL', 'Mean Actual Inventory']

# Display the filtered DataFrame
display(durabend_r12_aveinv_df)

,LABEL,Mean Actual Inventory
0,DURABEND R10,72.785833
1,DURABEND R12,740.805000
2,DURABEND R15,1875.064167
3,DURAFLEX R10,604.986667
4,DURAFLEX R12,55.591667
5,DURAFLEX R15,388.588333
6,DURAFLEX R23,190.660833


In [57]:
merged_df = pd.merge(demand_stats, durabend_r12_aveinv_df, on='LABEL', how='inner')

# Select the relevant columns
selected_columns = merged_df[['LABEL', 'Mean Actual Inventory']+ 
                             [f'AvgInv_{i}' for i in range(1, 5)]]

selected_columns = selected_columns.sort_values(by='Mean Actual Inventory', 
                                                ascending=False)

display(selected_columns)

,LABEL,Mean Actual Inventory,AvgInv_1,AvgInv_2,AvgInv_3,AvgInv_4
2,DURABEND R15,1875.064167,1939.903983,2007.489844,2271.010965,2370.199370
1,DURABEND R12,740.805000,870.682889,905.680979,1042.140526,1093.503410
3,DURAFLEX R10,604.986667,181.195532,193.848742,243.184326,261.754064
5,DURAFLEX R15,388.588333,142.119979,153.067850,195.754222,211.821219
6,DURAFLEX R23,190.660833,83.504091,89.776021,114.230633,123.435261
0,DURABEND R10,72.785833,26.686487,28.489906,35.521536,38.168217
4,DURAFLEX R12,55.591667,25.667769,27.230788,33.325088,35.618960


In [58]:
# Check if Mean Actual Inventory is within the range of AvgInv_1 and AvgInv_4
merged_df['Within Range'] = merged_df.apply(lambda row: row['AvgInv_1'] 
                                            <= row['Mean Actual Inventory'] 
                                            <= row['AvgInv_4'], axis=1)

# Calculate the difference between Mean Actual Inventory and AvgInv_1
merged_df['Difference'] = merged_df['Mean Actual Inventory'] - merged_df['AvgInv_1']

merged_df['Percent Difference'] = merged_df.apply(lambda row: f"{abs((row['Difference'] / row['AvgInv_1']) * 100):.2f}%", axis=1)

# Determine if the Mean Actual Inventory is above or below AvgInv_1
merged_df['Above or Below'] = merged_df.apply(lambda row: 'Above' 
                                              if row['Difference'] > 0 
                                              else 'Below', axis=1)

# selected_columns = selected_columns.sort_values(by='Percent Difference', ascending=False)

# Display the updated DataFrame
display(merged_df[['LABEL', 'Mean Actual Inventory', 'Within Range', 
                   'Percent Difference', 'Above or Below']])

,LABEL,Mean Actual Inventory,Within Range,Percent Difference,Above or Below
0,DURABEND R10,72.785833,False,172.74%,Above
1,DURABEND R12,740.805000,False,14.92%,Below
2,DURABEND R15,1875.064167,False,3.34%,Below
3,DURAFLEX R10,604.986667,False,233.89%,Above
4,DURAFLEX R12,55.591667,False,116.58%,Above
5,DURAFLEX R15,388.588333,False,173.42%,Above
6,DURAFLEX R23,190.660833,False,128.33%,Above


---

## Comparison & Reccomendations

Notice that none of the given mean inventory is actually within the range idealized by any of our 4 scenarios.

The greatest offenders, on a percentage basis, is `bend R10` and `flex R10`, which both are stocked with more than 150% surplus. 

Steelworks should reduce they're inventories for all SKUs except `Bend R12` and `Bend R15`. These should be increased by about 10%.

---

# 2) Luminex Solutions (LS) Inventory Management Problem

Luminex Solutions (LS) is a manufacturer of various high-end floor lamps. LS 
uses **180 LED bulbs** in its production lines each day (assume **360 days in a year**). 
LS sources the bulbs from an electronics supplier at a price of **$2 per bulb**. 
The supplier has a **lead time of 2 months** and charges a **shipping fee of $200 per order**. 
LS applies an **inventory carrying cost rate of 30%** for the LED bulbs.

## Bulk Order Discounts
The supplier has recently introduced **bulk order discounts** based on the order quantity:
- If LS orders **6,000 bulbs or fewer** at a time, the price remains **$2 per bulb**.
- For orders **between 6,001 and 8,000** bulbs, the price drops to **$1.75 per bulb**.
- For orders **exceeding 8,000 bulbs**, the price is reduced to **$1.50 per bulb**.

## Questions

### (a) How many bulbs should LS order each time?

---

I'm assuming this matches the **$(Q,r)$ Policy** Formulation

- **Order quantity**:  
  $$ Q = \sqrt{\frac{2K\mu}{h}} $$

Average total cost over time

$$\frac{KD}{Q} + \frac{hQ}{2} + cD$$

In [60]:
def avg_total_cost_rate(K,D,Q,h,c):
    return K*D/Q + h*Q/2 + c*D

def q1pta():
    daily_demand = 180
    mu_annual_demand = daily_demand *360

    def c_cost(order_quantity):
        if order_quantity > 0 and order_quantity <= 6_000:
            return 2
        if order_quantity > 6_000 and order_quantity <= 8_000:
            return 1.75
        if order_quantity > 8_000:
            return 1.5

    L_lead_time = 2 * 30                # Days
    K_fixed_order_cost = 200            # Shipping Fee
    perc_holding_cost = 0.3     # 30% of the inventory value


    

    for c, bound in zip((2, 1.75, 1.5), (6_000, 8_000, 8_001)):
        h = perc_holding_cost * c
        Q=(2*K_fixed_order_cost*mu_annual_demand/(h))**0.5
        
        if c_cost(Q) == c:
            print(f"Per unit Cost: {c}, Q: {Q}")
            print(f'Avg total cost over time is {avg_total_cost_rate(K_fixed_order_cost, mu_annual_demand, Q, h, c)}')
            print()
            continue

        print(f"Per unit Cost: {c}, Q: {Q} (not feasible)")
        Q = bound # this was choosen relative to the computed Q value restrospectively
        print(f"Per unit Cost: {c}, Q: {bound}")
        print(f'Avg total cost over time is {avg_total_cost_rate(K_fixed_order_cost, mu_annual_demand, Q, h, c)}')
        print()
q1pta()

Per unit Cost: 2, Q: 6572.670690061994 (not feasible)
Per unit Cost: 2, Q: 6000
Avg total cost over time is 133560.0

Per unit Cost: 1.75, Q: 7026.480525229438
Avg total cost over time is 117088.90227574545

Per unit Cost: 1.5, Q: 7589.466384404111 (not feasible)
Per unit Cost: 1.5, Q: 8001
Avg total cost over time is 100620.02252530934



---

### (b) What is the reorder point?

- **Reorder point**:  
  $$ r = L\mu + z_{\alpha} \sigma \sqrt{L} $$

We can't solve without knowing the variance of our demand, unless we have no variance; meaning we only care about the first term


In [61]:
def q2ptc():
    L_lead_time = 2 * 30                # Days
    mu_daily_demand = 180               # Units
    r = L_lead_time * mu_daily_demand
    print(f're-order at {r} units')

q2ptc()

re-order at 10800 units


### (c) Alternative Scenario
Assume that the supplier **does not offer any quantity discount** and its price remains **$2 per lamp** 
for **any order size**. However, the supplier is **increasing its shipment cost** for large orders.

- Specifically, for **orders of size 6,000 or less**, the order setup cost is the same as before, **$200 per order**.
- However, for orders **larger than 6,000**, the order setup cost increases to **$500**. 
  - This is mostly due to the fact that the supplier needs to send **two trucks** for orders larger than **6,000**.

**How many lamps should LS order from the supplier?**

In [66]:
def q1ptc():
    daily_demand = 180
    mu_annual_demand = daily_demand *360
    perc_holding_cost = 0.3     # 30% of the inventory value
    c_unit_cost = 2
    
    def K_fixed_order_cost(order_quantity):
        if order_quantity > 0 and order_quantity <= 6_000:
            return 200
        if order_quantity > 6_000:
            return 500

    for K, bound in zip((200, 500), (6_000, 6_001)):
        h = perc_holding_cost * c_unit_cost

        Q=(2*K*mu_annual_demand/(h))**0.5
        
        if K_fixed_order_cost(Q) == K:
            print(f"Fixed unit Cost: {K}, Q: {Q}")
            print(f'Avg total cost over time is {avg_total_cost_rate(K, mu_annual_demand, Q, h, c_unit_cost)}')
            print()
            continue

        print(f"Fixed unit Cost:{K}, Q: {Q} (Q not feasible)")

        Q = bound # this was choosen relative to the computed Q value restrospectively
        print(f"Q: {bound}")
        print(f'Avg total cost over time is {avg_total_cost_rate(K, mu_annual_demand, Q, h, c_unit_cost)}')
        print()

q1ptc()

Fixed unit Cost:200, Q: 6572.670690061994 (Q not feasible)
Q: 6000
Avg total cost over time is 133560.0

Fixed unit Cost: 500, Q: 10392.304845413264
Avg total cost over time is 135835.38290724796



---

# 3) CCAT Transmission Manufacturing Problem

CCAT is a manufacturer of construction vehicles such as loaders and bulldozers. Its transmission plant has the capacity of producing 10,000 transmissions each year. CCAT needs 4,000 transmissions each year. Each transmission is valued at approximately $6,000. 

- **Fixed setup cost** per batch: $50,000  
- **Annual inventory carrying rate** for the transmission: 20%

---

### Economic Production Quantity (EPQ) and Economic Production Lot (EPL)

## Diagram
A triangular inventory cycle is drawn, representing the replenishment and depletion of stock.

- The replenishment rate is **P**
- The demand rate is **D**
- The shaded area represents the production and depletion phases

## Optimal Order Quantity (Q*)
$$
Q^* = \sqrt{\frac{2KD}{h(1 - \frac{D}{P})} }
$$

## Per Unit Time Cost
$$
\text{Per unit time cost} = \frac{CC}{CT} = \frac{KD}{Q} + cD + \frac{hQ(P-D)}{2P}
$$

## Key Variables
- **K**: Setup cost
- **c**: Unit cost
- **Q**: Order quantity
- **D**: Demand rate
- **P**: Production rate
- **h**: Holding cost per unit per time

## Questions

### a) Economic Production Quantity (EPQ)
How many transmissions should CCAT produce in each run in order to minimize its annual production setup and inventory holding costs of transmissions?

Cycle Length
$
\text{Cycle Length} = \frac{Q}{D}
$


In [67]:
def q3pta():
    K = 50_000      # Fixed order cost
    c = 6_000        # to solve 
    D = 4_000        # Demand per year
    P_production_capacity = 10_000       # production capacity
    h_inventory_holding_cost = 0.2

    def get_optimal_q(K,D,h,P):
        return (2*K*D/(h*(1-D/P)))**0.5
    
    Q = get_optimal_q(K,D,h_inventory_holding_cost, P_production_capacity)
    print(f'Optimal Q: {Q} units')
    return Q

_ = q3pta()

Optimal Q: 57735.026918962576 units


### b) Production Run Length
What is the length of a production run?

Cycle Cost
$
\text{Cycle Cost} = K + cQ + h\frac{Q^2(P-D)}{2DP}
$

In [68]:
def q3ptb():
    Q = q3pta()
    D = 4_000        # Demand per year
    CT_cycle_time = Q/D
    print(f'Cycle Time: {CT_cycle_time} years')

q3ptb()

Optimal Q: 57735.026918962576 units
Cycle Time: 14.433756729740644 years


### c) Inventory Levels
- What is the **maximum inventory** of transmissions?
- What is the **average inventory** of transmissions?

 Maximum Inventory Level
$
H = \frac{Q}{P} (P - D)
$

Avg Inventory 
avg_inv $  = \frac{Q^2}{2D}-\frac{Q^2}{2P}$

In [69]:
def q3ptc():
    Q_quantity = q3pta()
    D_demand = 4_000        # Demand per year
    P_capacity = 10_000       # Production capacity
    avg_inv = Q_quantity**2/(2*D_demand) - Q_quantity**2/(2*P_capacity)
    print(f'Average Inventory: {avg_inv} units')

q3ptc()

Optimal Q: 57735.026918962576 units
Average Inventory: 250000.0 units


---

# 4) EOQ Model with Backorders

Consider the EOQ model that we discussed in class. In that model, we assumed that backorders are not allowed. Now, consider a variant where backorders are allowed. When a product is unique and popular, customers are willing to wait to receive the product if it is out of stock. Firms can take advantage of this to reduce their inventory cost and plan their inventory such that a certain fraction of customers is backordered.

Define:
- **p** as the backorder cost per unit per unit time.
- **x** as the fraction of demand that is backordered.

## Questions

### (a) Total Cost Function
What is the total cost per unit time function in terms of order quantity (**Q**), **x**, and the cost parameters? 

Consider a similar approach used in class:
1. Define the replenishment cycle.
2. Derive the expressions for the total cost and the length of a replenishment cycle.

---

1) Replenishment cycle is the length of time between two shipment of orders from the wholesaler to the retailer. Each replenishment cycle provides the retailer with Q units.


2) Replenishment Cycle Length is always 
- $CT = \frac{Q}{D}$ 
and cycle cost will be 
- $CC = K+cQ+h\frac{(Q(1-x))^2}{2D}+b\frac{(Qx)^2}{2D}$ 
where 

    - D is the rate of demand
    - K is fixed coster per order
    - c is unit cost per item
    - Q is the order quantity
    - h is the holding cost per unit per day
    - b is the backorder holding cost

Thus, total cost per unit time is derived to be:

$$\frac{CC}{CT}=\frac{K+cQ+h\frac{(Q(1-x))^2}{2D}+b\frac{(Qx)^2}{2D}}{\frac{Q}{D}}$$

### (b) Optimal Values of Q and x
What are the optimal values of:
- Order quantity (**Q**)?
- Fraction of backordered demand (**x**)?

In [70]:
import sympy as sp
from sympy import solve

def q4ptb():
    # Define the variables
    K, c, Q, h, x, D, b = sp.symbols('K c Q h x D b')

    # Define the cost function
    CC = K + c*Q + h*(Q*(1-x))**2/(2*D) + b*(Q*x)**2/(2*D)
    CT = Q/D
    total_cost = CC / CT

    # Take the derivative with respect to Q
    d_total_cost_dQ = sp.simplify(sp.diff(total_cost, Q))

    print('Derivative of the total cost with respect to Q:')
    # Display the derivatives
    sp.pprint(d_total_cost_dQ)


    print('\nOptimal Q:')
    # Solve for Q when d_total_cost_dQ = 0
    Q_solution = solve(d_total_cost_dQ, Q)
    Q_solution = (sp.simplify(Q_solution[1])) # We only care about the positive solution, Q > 0
    sp.pprint(Q_solution)

    # Take the second derivative with respect to Q
    d2_total_cost_dQ2 = sp.diff(d_total_cost_dQ, Q)
    
    print('\nSecond derivative of the total cost with respect to Q:')
    # Display the second derivatives
    sp.pprint(d2_total_cost_dQ2)
q4ptb()


Derivative of the total cost with respect to Q:
           2      2          
  D⋅K   b⋅x    h⋅x          h
- ─── + ──── + ──── - h⋅x + ─
   2     2      2           2
  Q                          

Optimal Q:
        _________________________
       ╱           D⋅K           
√2⋅   ╱  ─────────────────────── 
     ╱      2      2             
   ╲╱    b⋅x  + h⋅x  - 2⋅h⋅x + h 

Second derivative of the total cost with respect to Q:
2⋅D⋅K
─────
  3  
 Q   


Since the 2nd derivative is always positive, we know this is a global minima, and therefore our optimal Q is above.

---

### (c) Optimal Conditions for Backordering
1. When is it optimal to backorder?
2. Under what condition(s) is this model equivalent to the EOQ model?

---

1) Backorders can be defined as when x>0. Let's look at the total cost per unit time function w.r.t. x

In [71]:
import sympy as sp
from sympy import solve

def q4ptc():
    # Define the variables
    K, c, Q, h, x, D, b = sp.symbols('K c Q h x D b')

    # Define the cost function
    CC = K + c*Q + h*(Q*(1-x))**2/(2*D) + b*(Q*x)**2/(2*D)
    CT = Q/D
    total_cost = CC / CT

    # Take the derivative with respect to X
    d_total_cost_dx = sp.simplify(sp.diff(total_cost, x))

    print('Derivative of the total cost per unit time with respect to x:')
    # Display the derivatives
    sp.pprint(d_total_cost_dx)

    print('\nOptimal x:')
    # Solve for x when d_total_cost_dx = 0
    x_solution = solve(d_total_cost_dx, x)
    sp.pprint(x_solution)

    print('\nSecond derivative of the total cost per unit time with respect to x:')
    # Take the second derivative with respect to x
    d2_total_cost_dx2 = sp.diff(d_total_cost_dx, x)
    # Display the second derivatives
    sp.pprint(d2_total_cost_dx2)

q4ptc()

Derivative of the total cost per unit time with respect to x:
Q⋅(b⋅x + h⋅(x - 1))

Optimal x:
⎡  h  ⎤
⎢─────⎥
⎣b + h⎦

Second derivative of the total cost per unit time with respect to x:
Q⋅(b + h)


The 2nd derivative is positive; thus, we know this is a global minima. Optimal x is therefore at $\frac{h}{b+h}$

---